In [ ]:
import json
from pathlib import Path
from collections import Counter

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks", rc={'axes.formatter.limits': (-4, 5)}, font_scale=1.1)
figsize=(4.8*1/0.618,4.8)

In [ ]:
datasets = {
    "spider": ["spider", "spider_syn", "spider_realistic", "spider_dr"],
    "bird": ["bird"],
    "fiben": ["fiben"],
}
result_dirs = {
    "spider": Path("../results/test/silver-sun-65/k5ou2ykv"),
    "bird": Path("../results/test/happy-dew-65/ey4tt3n2"),
    "fiben": Path("../results/test/electric-donkey-66/sj2wkc2b"),
}
pds = []
N = 51
methods = ["DBCᴏᴘɪʟᴏᴛ", "DPR", "CRUSH$_{\mathrm{SXFMR}}$", "CRUSH$_{\mathrm{BM25}}$", "SXFMR", "BM25"]
for method in methods:
    scores = {
        "P": Counter(),
        "R": Counter(),
        "n": Counter(),
    }
    for dataset, tests in datasets.items():
        for test in tests:
            if method == "DPR":
                result_file = Path("../results") / "retrieval" / f"{test}_dense_true.json"
            elif method == "CRUSH$_{\mathrm{BM25}}$":
                result_file = Path("../results") / "retrieval" / f"crush4sql_{test}_sparse_false.json"
            elif method == "CRUSH$_{\mathrm{SXFMR}}$":
                result_file = Path("../results") / "retrieval" / f"crush4sql_{test}_dense_false.json"
            elif method == "SXFMR":
                result_file = Path("../results") / "retrieval" / f"{test}_dense_false.json"
            elif method == "BM25":
                result_file = Path("../results") / "retrieval" / f"{test}_sparse_false.json"
            elif method == "DBCᴏᴘɪʟᴏᴛ":
                result_file = result_dirs[dataset] / f'{test.replace(dataset, "test")}.json'

            with result_file.open() as file:
                results = json.load(file)

            for it in results:
                gold_schema = it["schema"]
                gold_tables = [
                    f'{gold_schema["database"]}.{tbl["name"]}'
                    for tbl in gold_schema["metadata"]
                ]
                gold = set(gold_tables)

                pred_dbs = []
                pred_tables = []
                for pred in it["pred_schemas"]:
                    pred_dbs.append(pred["database"])
                    pred_tables.extend(
                        f'{pred["database"]}.{tbl}'
                        for tbl in pred["tables"]
                    )

                for k in range(1, N):
                    cands = set(pred_tables[:k])
                    tp = len(cands & gold)
                    p_k = tp / len(cands) if len(cands) else 0
                    r_k = tp / len(gold) if len(gold) else 0
                    scores["P"][k] += p_k
                    scores["R"][k] += r_k
                    scores["n"][k] += 1

            recalls = []
            K = []
            for k in range(1, N):
                recalls.append(scores["R"][k] / scores["n"][k])
                K.append(k)
    
            pds.append(pd.DataFrame({"Recall@$k$": recalls, "$k$": K, "Method": [method]*len(precisions)}))

pr = pd.concat(pds, ignore_index=True)
plt.figure(figsize=figsize)
sns.lineplot(data=pr, x="$k$", y="Recall@$k$", hue="Method", style="Method")
sns.despine()
plt.tight_layout()
plt.savefig('recall_curve.pdf')

In [ ]:
from sklearn.metrics import auc

datasets = {
    "spider": ["spider", "spider_syn", "spider_realistic", "spider_dr"],
    "bird": ["bird"],
    "fiben": ["fiben"],
}
result_dirs = {
    "spider": Path("../results/test/silver-sun-65/k5ou2ykv"),
    "bird": Path("../results/test/happy-dew-65/ey4tt3n2"),
    "fiben": Path("../results/test/electric-donkey-66/sj2wkc2b"),
}
pds = []
N = 16
methods = ["DBCᴏᴘɪʟᴏᴛ", "DPR", "CRUSH$_{\mathrm{SXFMR}}$", "CRUSH$_{\mathrm{BM25}}$", "SXFMR", "BM25"]
for method in methods:
    scores = {
        "AP": Counter(),
        "n": Counter(),
    }
    for dataset, tests in datasets.items():
        with Path(f"../data/{dataset}/schemas.json").open("r") as f:
            schemas = json.load(f)
            
        for test in tests:
            if method == "DPR":
                result_file = Path("../results") / "retrieval" / f"{test}_dense_true.json"
            elif method == "CRUSH$_{\mathrm{BM25}}$":
                result_file = Path("../results") / "retrieval" / f"crush4sql_{test}_sparse_false.json"
            elif method == "CRUSH$_{\mathrm{SXFMR}}$":
                result_file = Path("../results") / "retrieval" / f"crush4sql_{test}_dense_false.json"
            elif method == "SXFMR":
                result_file = Path("../results") / "retrieval" / f"{test}_dense_false.json"
            elif method == "BM25":
                result_file = Path("../results") / "retrieval" / f"{test}_sparse_false.json"
            elif method == "DBCᴏᴘɪʟᴏᴛ":
                result_file = result_dirs[dataset] / f'{test.replace(dataset, "test")}.json'

            with result_file.open() as file:
                results = json.load(file)

            for it in results:
                gold_schema = it["schema"]
                gold_tables = [
                    f'{gold_schema["database"]}.{tbl["name"]}'
                    for tbl in gold_schema["metadata"]
                ]
                gold = set(gold_tables)

                pred_dbs = []
                pred_tables = []
                for pred in it["pred_schemas"]:
                    pred_dbs.append(pred["database"])
                    pred_tables.extend(
                        f'{pred["database"]}.{tbl}'
                        for tbl in pred["tables"]
                    )

                Ps, Rs = [1], [0]
                for k in range(1, N):
                    cands = set(pred_tables[:k])
                    tp = len(cands & gold)
                    p_k = tp / len(cands) if len(cands) else 0
                    r_k = tp / len(gold) if len(gold) else 0

                    Ps.append(p_k)
                    Rs.append(r_k)
                    
                AP = auc(Rs, Ps)
                scores["AP"][len(schemas[gold_schema["database"]])] += AP
                scores["n"][len(schemas[gold_schema["database"]])] += 1
                
    mAP = []
    K = []
    for k in range(20):
        if scores["n"][k] != 0:
            mAP.append(scores["AP"][k] / scores["n"][k])
            K.append(k)

    pds.append(pd.DataFrame({"mAP": mAP, "Table Number": K, "Method": [method]*len(K)}))

pr = pd.concat(pds, ignore_index=True)
plt.figure(figsize=figsize)
sns.lineplot(data=pr, x="Table Number", y="mAP", hue="Method", style="Method", marker='o')
sns.despine()
plt.tight_layout()
plt.savefig('table_number.pdf')

In [ ]:
# Given data for Database Recall@1 and Table Recall@5
data_recall_1 = {
    "Dataset": ['Spider', 'Bird', 'Fiben'],
    "5000": [50.68, 70.99, 100],
    "10000": [72.15, 79.79, 100],
    "20000": [77.66, 85.07, 100],
    "40000": [82.01, 88.14, 100],
    "80000": [85.29, 89.11, 100],
    "100000": [84.43, 88.85, 100]
}

data_recall_5 = {
    "Dataset": ['Spider', 'Bird', 'Fiben'],
    "5000": [33.1, 40.9, 22.7],
    "10000": [58.85, 54.23, 30.29],
    "20000": [68.12, 61.67, 35.9],
    "40000": [72.72, 65.7, 33.71],
    "80000": [77.44, 70.09, 34.28],
    "100000": [76.55, 70.43, 35.99]
}

# Convert dictionaries to pandas DataFrames for plotting
df_recall_1 = pd.DataFrame(data_recall_1).melt(id_vars=['Dataset'], var_name='# Synthetic Data', value_name='Recall@1')
df_recall_5 = pd.DataFrame(data_recall_5).melt(id_vars=['Dataset'], var_name='# Synthetic Data', value_name='Recall@5')

df_recall_1 = df_recall_1.astype({"# Synthetic Data": int})
df_recall_5 = df_recall_5.astype({"# Synthetic Data": int})

# Now let's plot using seaborn
plt.tight_layout()
fig, axs = plt.subplots(1, 2, figsize=(9.6,4.8), constrained_layout=True)

# Plot recall@1
sns.lineplot(x='# Synthetic Data', y='Recall@1', hue='Dataset', data=df_recall_1, marker='o', ax=axs[0])
axs[0].set_title('Database')

# Plot recall@5
sns.lineplot(x='# Synthetic Data', y='Recall@5', hue='Dataset', data=df_recall_5, marker='o', ax=axs[1])
axs[1].set_title('Table')

sns.despine()
plt.savefig('data_number.pdf')

In [ ]:
# Now let's plot using seaborn
plt.tight_layout()
fig, axs = plt.subplots(1, 2, figsize=(4.8*2*1.2,4.8), constrained_layout=True)

datasets = {
    "spider": ["spider", "spider_syn", "spider_realistic", "spider_dr"],
    "bird": ["bird"],
    "fiben": ["fiben"],
}
result_dirs = {
    "spider": Path("../results/test/silver-sun-65/k5ou2ykv"),
    "bird": Path("../results/test/happy-dew-65/ey4tt3n2"),
    "fiben": Path("../results/test/electric-donkey-66/sj2wkc2b"),
}
pds = []
N = 51
methods = ["DBCᴏᴘɪʟᴏᴛ", "DPR", "CRUSH$_{\mathrm{SXFMR}}$", "CRUSH$_{\mathrm{BM25}}$", "SXFMR", "BM25"]
for method in methods:
    scores = {
        "P": Counter(),
        "R": Counter(),
        "n": Counter(),
    }
    for dataset, tests in datasets.items():
        for test in tests:
            if method == "DPR":
                result_file = Path("../results") / "retrieval" / f"{test}_dense_true.json"
            elif method == "CRUSH$_{\mathrm{BM25}}$":
                result_file = Path("../results") / "retrieval" / f"crush4sql_{test}_sparse_false.json"
            elif method == "CRUSH$_{\mathrm{SXFMR}}$":
                result_file = Path("../results") / "retrieval" / f"crush4sql_{test}_dense_false.json"
            elif method == "SXFMR":
                result_file = Path("../results") / "retrieval" / f"{test}_dense_false.json"
            elif method == "BM25":
                result_file = Path("../results") / "retrieval" / f"{test}_sparse_false.json"
            elif method == "DBCᴏᴘɪʟᴏᴛ":
                result_file = result_dirs[dataset] / f'{test.replace(dataset, "test")}.json'

            with result_file.open() as file:
                results = json.load(file)

            for it in results:
                gold_schema = it["schema"]
                gold_tables = [
                    f'{gold_schema["database"]}.{tbl["name"]}'
                    for tbl in gold_schema["metadata"]
                ]
                gold = set(gold_tables)

                pred_dbs = []
                pred_tables = []
                for pred in it["pred_schemas"]:
                    pred_dbs.append(pred["database"])
                    pred_tables.extend(
                        f'{pred["database"]}.{tbl}'
                        for tbl in pred["tables"]
                    )

                for k in range(1, N):
                    cands = set(pred_tables[:k])
                    tp = len(cands & gold)
                    p_k = tp / len(cands) if len(cands) else 0
                    r_k = tp / len(gold) if len(gold) else 0
                    scores["P"][k] += p_k
                    scores["R"][k] += r_k
                    scores["n"][k] += 1

            recalls = []
            K = []
            for k in range(1, N):
                recalls.append(scores["R"][k] / scores["n"][k])
                K.append(k)
    
            pds.append(pd.DataFrame({"Recall@$k$": recalls, "$k$": K, "Method": [method]*len(precisions)}))

pr = pd.concat(pds, ignore_index=True)
sns.lineplot(data=pr, x="$k$", y="Recall@$k$", hue="Method", style="Method", ax=axs[1])

from sklearn.metrics import auc

datasets = {
    "spider": ["spider", "spider_syn", "spider_realistic", "spider_dr"],
    "bird": ["bird"],
    "fiben": ["fiben"],
}
result_dirs = {
    "spider": Path("../results/test/silver-sun-65/k5ou2ykv"),
    "bird": Path("../results/test/happy-dew-65/ey4tt3n2"),
    "fiben": Path("../results/test/electric-donkey-66/sj2wkc2b"),
}
pds = []
N = 16
methods = ["DBCᴏᴘɪʟᴏᴛ", "DPR", "CRUSH$_{\mathrm{SXFMR}}$", "CRUSH$_{\mathrm{BM25}}$", "SXFMR", "BM25"]
for method in methods:
    scores = {
        "AP": Counter(),
        "n": Counter(),
    }
    for dataset, tests in datasets.items():
        with Path(f"../data/{dataset}/schemas.json").open("r") as f:
            schemas = json.load(f)
            
        for test in tests:
            if method == "DPR":
                result_file = Path("../results") / "retrieval" / f"{test}_dense_true.json"
            elif method == "CRUSH$_{\mathrm{BM25}}$":
                result_file = Path("../results") / "retrieval" / f"crush4sql_{test}_sparse_false.json"
            elif method == "CRUSH$_{\mathrm{SXFMR}}$":
                result_file = Path("../results") / "retrieval" / f"crush4sql_{test}_dense_false.json"
            elif method == "SXFMR":
                result_file = Path("../results") / "retrieval" / f"{test}_dense_false.json"
            elif method == "BM25":
                result_file = Path("../results") / "retrieval" / f"{test}_sparse_false.json"
            elif method == "DBCᴏᴘɪʟᴏᴛ":
                result_file = result_dirs[dataset] / f'{test.replace(dataset, "test")}.json'

            with result_file.open() as file:
                results = json.load(file)

            for it in results:
                gold_schema = it["schema"]
                gold_tables = [
                    f'{gold_schema["database"]}.{tbl["name"]}'
                    for tbl in gold_schema["metadata"]
                ]
                gold = set(gold_tables)

                pred_dbs = []
                pred_tables = []
                for pred in it["pred_schemas"]:
                    pred_dbs.append(pred["database"])
                    pred_tables.extend(
                        f'{pred["database"]}.{tbl}'
                        for tbl in pred["tables"]
                    )

                Ps, Rs = [1], [0]
                for k in range(1, N):
                    cands = set(pred_tables[:k])
                    tp = len(cands & gold)
                    p_k = tp / len(cands) if len(cands) else 0
                    r_k = tp / len(gold) if len(gold) else 0

                    Ps.append(p_k)
                    Rs.append(r_k)
                    
                AP = auc(Rs, Ps)
                scores["AP"][len(schemas[gold_schema["database"]])] += AP
                scores["n"][len(schemas[gold_schema["database"]])] += 1
                
    mAP = []
    K = []
    for k in range(20):
        if scores["n"][k] != 0:
            mAP.append(scores["AP"][k] / scores["n"][k])
            K.append(k)

    pds.append(pd.DataFrame({"mAP": mAP, "Table Number": K, "Method": [method]*len(K)}))

pr = pd.concat(pds, ignore_index=True)
sns.lineplot(data=pr, x="Table Number", y="mAP", hue="Method", style="Method", marker='o', ax=axs[0])
axs[0].set_title('(a)', fontsize=18)
axs[1].set_title('(b)', fontsize=18)

sns.despine()
plt.savefig('table_number.pdf')